In [ ]:
import os
os.chdir(r'C:\Users\WeiChuanzheng\Desktop\T2T_FC6\orthofinder_rename')

In [ ]:
samples = [i.replace('.fa','') for i in os.listdir('01.protein/')]

In [ ]:
len(samples)

In [ ]:
directories = ['02.cds', '02.protein', '03.cds', '03.protein', '04.cds', '04.protein']
for dir in directories:
    os.makedirs(dir, exist_ok=True)

In [ ]:
def FastaToDict(file):
    fasta = {}
    with open(file, 'r')as f:
        lines = f.readlines()
    for line in lines:
        if line[0] == '>':
            key = line.strip()
            fasta[key] = []
        else:
            fasta[key].append(line.strip())
    for key,value in list(fasta.items()):
        fasta[key] = ''.join(value)
    return fasta

In [ ]:
def rename_protein(sample):
    seq = FastaToDict(f'01.protein/{sample}.fa')
    count = 0
    new_cds_lines = []
    match_lines = []
    for k, v in seq.items():
        k = k.split()[0]
        count += 1
        name = f"S{str(count).zfill(6)}"
        match_lines.append(f"{k[1:]}\t{name}\n")
        new_cds_lines.append(f">{name}\n")
        for i in range(0, len(v), 70):
            new_cds_lines.append(f"{v[i:i+70].rstrip('*')}\n")

    with open(f'02.protein/{sample}.match.txt', 'w') as f:
        f.writelines(match_lines)

    with open(f'02.protein/{sample}.fa', 'w') as f:
        f.writelines(new_cds_lines)

    print(count)

In [ ]:
def rename_cds(sample):
    seq = FastaToDict(f'01.cds/{sample}.cds.fa')
    count = 0
    new_cds_lines = []
    match_lines = []
    for k, v in seq.items():
        k = k.split()[0]
        count += 1
        name = f"S{str(count).zfill(6)}"
        match_lines.append(f"{k[1:]}\t{name}\n")
        new_cds_lines.append(f">{name}\n")
        for i in range(0, len(v), 70):
            new_cds_lines.append(f"{v[i:i+70].rstrip('*')}\n")

    with open(f'02.cds/{sample}.match.txt', 'w') as f:
        f.writelines(match_lines)

    with open(f'02.cds/{sample}.cds.fa', 'w') as f:
        f.writelines(new_cds_lines)

    print(count)

In [ ]:
def delate_point_protein(sample):
    seq = FastaToDict('02.protein/{}.fa'.format(sample))
    with open('03.protein/{}.fa'.format(sample), 'w')as f:
        count = 0
        for k,v in seq.items():
            if '.' in v:continue
            if '*' in v:continue
            count += 1
            f.write(f"{k}\n")
            for i in range(0, len(v), 70):
                f.write(f"{v[i:i+70]}\n")
        print(count)

In [ ]:
def delate_point_cds(sample):
    seq1 = FastaToDict('03.protein/{}.fa'.format(sample))
    seq2 = FastaToDict('02.cds/{}.cds.fa'.format(sample))
    with open('03.cds/{}.cds.fa'.format(sample), 'w')as f:
        count = 0
        for k,v in seq1.items():
            count += 1
            f.write(f"{k}\n")
            s = seq2[k]
            for i in range(0, len(s), 70):
                f.write(f"{s[i:i+70]}\n")
        print(count)

In [ ]:
rename_protein('Fhisp')
rename_cds('Fhisp')
delate_point_protein('Fhisp')
delate_point_cds('Fhisp')

In [ ]:
for sample in samples:
    print(sample)
    rename_protein(sample)
    rename_cds(sample)
    delate_point_protein(sample)
    delate_point_cds(sample)

In [ ]:
with open('fastix.sh', 'w')as f:
    for sample in samples:
        f.write(f"fastix -p \"{sample}#\" 03.cds/{sample}.cds.fa > 04.cds/{sample}.cds.fa\n")
        f.write(f"fastix -p \"{sample}#\" 03.protein/{sample}.fa > 04.protein/{sample}.fa\n")